In [1]:
import numpy as np
import pandas as pd

import os
os.listdir('../input/imet-2020-fgvc7')

['train.csv', 'train', 'test', 'sample_submission.csv', 'labels.csv']

In [2]:
submission = pd.read_csv('../input/imet-2020-fgvc7/sample_submission.csv')
submission.head()

,id,attribute_ids
0,00011f01965f141f5d1eea6592fa9862,0 1 2
1,00014abc91ed3e4bf1663fde8136fe80,0 1 2
2,0002e2054e303badc1a33463f6fb7973,0 1 2
3,0002e8f35f85f28bebfb28f2a627dc4d,0 1 2
4,00082dfc0de78506f96104bc05eb5a49,0 1 2


In [3]:
import random
from collections import OrderedDict
import warnings
import cv2
from PIL import Image
from functools import partial

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torchvision.models as M
from albumentations import Compose, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip
from albumentations.pytorch import ToTensorV2

from tqdm import tqdm
from sklearn.metrics import fbeta_score
from sklearn.exceptions import UndefinedMetricWarning

In [4]:
def seed_torch(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch()

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
N_CLASSES = 3474


class TrainDataset(Dataset):
    def __init__(self, df, labels, transform=None):
        self.df = df
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df['id'].values[idx]
        file_path = f'../input/imet-2020-fgvc7/train/{file_name}.png'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        label = self.labels.values[idx]
        target = torch.zeros(N_CLASSES)
        for cls in label.split():
            target[int(cls)] = 1
        
        return image, target
    

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df['id'].values[idx]
        file_path = f'../input/imet-2020-fgvc7/test/{file_name}.png'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image

In [7]:
HEIGHT = 128
WIDTH = 128


def get_transforms(data):
    assert data in ('train', 'valid')
    
    if data == 'train':
        return Compose([
            #Resize(HEIGHT, WIDTH),
            RandomResizedCrop(HEIGHT, WIDTH),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
            #Resize(HEIGHT, WIDTH),
            RandomCrop(256, 256),
            HorizontalFlip(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
            
        ])

In [8]:
batch_size = 128

test_dataset = TestDataset(submission, transform=get_transforms('valid'))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
class AvgPool(torch.nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])


def create_net(net_cls, pretrained: bool):
    if pretrained:
        net = net_cls()
        model_name = net_cls.__name__
        weights_path = f'../input/{model_name}/{model_name}.pth'
        net.load_state_dict(torch.load(weights_path))
    else:
        net = net_cls(pretrained=pretrained)
    return net


class ResNet(torch.nn.Module):
    def __init__(self, num_classes, pretrained=False, net_cls=M.resnet50):
        super().__init__()
        self.net = create_net(net_cls, pretrained=pretrained)
        self.net.avgpool = AvgPool()
        self.net.fc = torch.nn.Linear(self.net.fc.in_features, num_classes)

    def fresh_params(self):
        return self.net.fc.parameters()

    def forward(self, x):
        return self.net(x)


resnet50 = partial(ResNet, net_cls=M.resnet50)
resnet101 = partial(ResNet, net_cls=M.resnet101)

In [10]:
criterion = nn.BCEWithLogitsLoss(reduction='none')
model_101 = resnet101(num_classes=N_CLASSES, pretrained=True)
model_fold0 = resnet50(num_classes=N_CLASSES, pretrained=True)
model_fold1 = resnet50(num_classes=N_CLASSES, pretrained=True)
model_fold2 = resnet50(num_classes=N_CLASSES, pretrained=True)

In [11]:
def load_model(model, path):
    print(path)
    state = torch.load(str(path))
    state_model = state['model']
    wrong_keys = {
        'net.fc.1.weight': 'net.fc.weight', 
        'net.fc.1.bias': 'net.fc.bias'
    }
    state['model'] = OrderedDict((wrong_keys.get(k) if k in wrong_keys else k, v) 
                                 for k, v in state_model.items())
    model.load_state_dict(state['model'])
    print('Loaded model from epoch {epoch}, step {step:,}'.format(**state))
    return state

In [12]:
from contextlib import contextmanager
import time

def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('Herbarium')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'train.log'
LOGGER = init_logger(LOG_FILE)

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

# model101

In [13]:
load_model(model_101, '../input/bestforresnet101/my-best-model2.pt')

with timer('inference'):
    model_101.to(device) 
    preds_model101 = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))

    for i, images in tk0:
        images = images.to(device)
        with torch.no_grad():
            y_preds = model_101(images)
        preds_model101.append(torch.sigmoid(y_preds).to('cpu').numpy())     
    
np_model101=np.concatenate(preds_model101)

../input/bestforresnet101/my-best-model2.pt


2020-05-14 19:06:39,817 INFO [inference] start
  0%|          | 0/203 [00:00<?, ?it/s]

Loaded model from epoch 19, step 31,986


100%|██████████| 203/203 [05:59<00:00,  1.77s/it]
2020-05-14 19:12:39,486 INFO [inference] done in 360 s.


# model_fold0

In [14]:
load_model(model_fold0, '../input/bestforresnet50/best-model0.pt')

with timer('inference'):
    model_fold0.to(device) 
    preds_model_fold0 = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))

    for i, images in tk0:
        images = images.to(device)
        with torch.no_grad():
            y_preds = model_fold0(images)
        preds_model_fold0.append(torch.sigmoid(y_preds).to('cpu').numpy())   
    
np_fold0=np.concatenate(preds_model_fold0)

../input/bestforresnet50/best-model0.pt


2020-05-14 19:12:39,825 INFO [inference] start
  0%|          | 0/203 [00:00<?, ?it/s]

Loaded model from epoch 20, step 33,763


100%|██████████| 203/203 [03:53<00:00,  1.15s/it]
2020-05-14 19:16:33,657 INFO [inference] done in 234 s.


# model_fold1

In [15]:
load_model(model_fold1, '../input/bestforresnet50/best-model1.pt')

with timer('inference'):
    model_fold1.to(device) 
    preds_model_fold1 = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))

    for i, images in tk0:
        images = images.to(device)
        with torch.no_grad():
            y_preds = model_fold1(images)
        preds_model_fold1.append(torch.sigmoid(y_preds).to('cpu').numpy())      
        
np_fold1=np.concatenate(preds_model_fold1)

../input/bestforresnet50/best-model1.pt


2020-05-14 19:16:34,074 INFO [inference] start
  0%|          | 0/203 [00:00<?, ?it/s]

Loaded model from epoch 20, step 33,744


100%|██████████| 203/203 [03:51<00:00,  1.14s/it]
2020-05-14 19:20:25,672 INFO [inference] done in 232 s.


# model_fold2

In [16]:
load_model(model_fold2, '../input/bestforresnet50/best-model2.pt')

with timer('inference'):
    model_fold2.to(device) 
    preds_model_fold2 = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))

    for i, images in tk0:
        images = images.to(device)
        with torch.no_grad():
            y_preds = model_fold2(images)
        preds_model_fold2.append(torch.sigmoid(y_preds).to('cpu').numpy())      
        
np_fold2=np.concatenate(preds_model_fold2)

../input/bestforresnet50/best-model2.pt


2020-05-14 19:20:26,066 INFO [inference] start
  0%|          | 0/203 [00:00<?, ?it/s]

Loaded model from epoch 16, step 26,655


100%|██████████| 203/203 [03:50<00:00,  1.14s/it]
2020-05-14 19:24:16,583 INFO [inference] done in 231 s.


In [17]:
avg_pred = np.mean( np.array([np_model101, np_fold0, np_fold1, np_fold2]), axis=0 )

In [18]:
threshold = 0.13
predictions = avg_pred > threshold

for i, row in enumerate(predictions):
    ids = np.nonzero(row)[0]
    submission.iloc[i].attribute_ids = ' '.join([str(x) for x in ids])
    
submission.to_csv('submission.csv', index=False)
submission.head()

,id,attribute_ids
0,00011f01965f141f5d1eea6592fa9862,149 370 2493
1,00014abc91ed3e4bf1663fde8136fe80,233 783 2103 2436 3433
2,0002e2054e303badc1a33463f6fb7973,149 370 2493
3,0002e8f35f85f28bebfb28f2a627dc4d,784 1131 3170 3364 3392 3456
4,00082dfc0de78506f96104bc05eb5a49,641 784 2557 3334
